In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.figure12 import Figure12
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# env = Figure12(5, 5, (4, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [3]:
# adam_and_eve = []
# env.reset()
# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# # init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# # init_learner = Learner(uuid.uuid4(), init_rule)
# # (init_team.learners).append(init_learner)
# # init_team.mostRecent = init_learner
# adam_and_eve.append(init_team)

# for adam in adam_and_eve:
#     print(adam.learners)
#     print('------')
#     adam.init_search(env, [2, 3])
#     print(adam.learners)
#     for learner in adam.learners:
#         print(learner.program.rule.action_set)
#         print(learner.program.rule.region)

In [4]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 1, 1, 0)
init_team.init_search(env, [2, 3])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
#     win = adam.search(env)
    data = [adam, win]
    searchManager.evaluate_team(data)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    if win:
        print(f'got to --> gen {gen+1}')
        break
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
#     if gen == 0:
#         for adam in adam_and_eve:
#             win = adam.search(env)
#             data = [adam, win]
#             searchManager.evaluate_team(data)
#     else:
    for parent in searchManager.teamPool:
        # only run if the team has not yet won
        if parent not in searchManager.winners:
            print(f'Parent: {parent.id}')
            env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search(env)
            print(f'Child: {child.id}')
            print('Start Child Regions --------')
            for learner in child.learners:
                print(learner.program.rule.region)
            print('End Child Regions --------')
            print(f'Child Fitness: {child.fitness}') 
            toEvaluateAfterGen.append([child, win])
        else:
            print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

new step-------
2
(0, 0)
new step-------
2
(0, 1)
new step-------
2
(0, 2)
new step-------
2
(0, 3)
new step-------
2
(0, 4)
new step-------
2
(0, 4)
new step-------
3
(0, 3)
new step-------
3
(0, 2)
new step-------
3
(0, 1)
new step-------
3
(0, 0)
new step-------
3
(0, 0)
[0, 0, 0, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: fb13bae4-bb36-42a8-afd2-d6c140b8e789
Start: (0, 0)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
Child: 9ad97eb3-465e-420e-b589-a1557ee1bf2f
Start Child Regions 

Child: 13422bc2-2fcc-4c8c-95f9-ad4fa6192d85
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 3]
End Child Regions --------
Child Fitness: 4.43
Parent: ca782515-9a0b-4d5e-9fb8-e42bd6f00ad1
Start: (4, 0)
New real step: 
0
(4, 0)
Backtrackregioncehck: [0, 4, 1, 4]
New real step: 
1
(3, 2)
New real step: 
1
(2, 2)
New real step: 
1
(1, 2)
New real step: 
1
(0, 2)
New real step: 
1
(0, 2)
Child: b8b9587f-cb37-4cb4-a062-6f6a4337cd28
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
End Child Regions --------
Child Fitness: 4.819999999999999
Parent: 63392b1c-7dd1-498b-9e0a-0fcdd20dab54
Start: (4, 4)
New real step: 
1
(3, 4)
New real step: 
1
(2, 4)
win!
Child: a9240e9a-47f6-4686-b143-095cbbeeb557
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 

In [5]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1004.63
Learners:
Region: [0, 0, 4, 4] --> Action: [2, 3]
Region: [1, 4, 0, 0] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 4] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 0, 3] --> Action: [2, 3]
Region: [1, 4, 2, 4] --> Action: [0, 1]
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$


In [6]:
team = searchManager.winners[0]
for learner in team.learners:
    print(learner.program.rule.region)

for i in range(5):
    print(f'Epoch: {i+1} --------')
    env.reset()
    win = False
    selected_learner = team.select_learner(env)
    while not win:
        win, needs_update, reward, winning_action = team.q_evaluation(env, selected_learner)
        print(reward)
        if not win:
            selected_learner = team.select_learner(env)
            team.transition_update(reward, winning_action, needs_update, selected_learner)

    team.final_update(reward, winning_action, needs_update)

print(win)
for learner in team.learners:
    print(f'{learner.program.rule.region}')
    for i in range(len(learner.program.rule.action_set)):
          print(f'{learner.program.rule.action_set[i]} --> {learner.program.rule.value_set[i]}')

[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 3]
[1, 4, 2, 4]
Epoch: 1 --------
[0, 0, 0, 1]
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 2
0.1
[0, 4, 3, 4]
[1, 4, 2, 4]
Winning action: 3
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 1
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 1
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 1
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 1
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 2
0.1
[0, 4, 3, 4]
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
Epoch: 2 --------
[0, 0, 0, 1]
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
[1, 2, 0, 3]
Winning action: 0
0.1
[1, 2, 1, 4]
[0, 4, 0, 3]
Winning action: 2
0.1
[0, 4, 3, 